<a href="https://colab.research.google.com/github/EdwardMMiller/Data-Mining-Project---Fall-2022/blob/main/New_Classifier_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NEWS CLASSIFIER**

## DATA MINING - FALL 2022
## CUNY GRADUATE CENTER
## Edward Miller

## Overview

Over the past few decades, news sources have drastically shifted from print media to
online sites due to the widespread adoption of the internet by the general public. As a
consequence of this shift, it has become easier for news articles from untrustworthy
sources to pass themselves as real, and cause disinformation to spread rapidly. This
disinformation can cause widespread damage and is often used to push false narratives to
benefit a political party or a government. It has already caused harm to democratic
institutions by eroding trust in media sources, as well as motivating extreme actions based
on incorrect information, such as the Capitol Riots on January 6, 2021 . Therefore, it is now
more important than ever to develop sound methods for determining whether a news
story is real, or not. The intent of this project will be to use deep learning in order to train a
model that can correctly classify a list of news stories as real or not real.

## Goals
1. Train a deep learning model for binary classification to correctly classify whether a
news story is real
2. Test Trained Model on a dataset to determine how well it differentiates between the
two classes, and report findings.

## Dataset
The dataset, as well as the direct inspiration for this project, comes from the following kaggle
website, and the author of the code has some good insights that will be reused here as well.

https://www.kaggle.com/code/urkchar/determine-if-news-is-fake-or-real/notebook

Progress will be updated through Github here:

https://github.com/EdwardMMiller/Data-Mining-Project---Fall-2022/new/main

In [1]:
!pip install pandas
!pip install numpy
!pip install tensorflow
!pip install keras
!pip3 install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging
import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

# libraries needed for 
# universal sentence encoder  

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV, train_test_split
# importing sklearn packages

import re
# importing text library

import zipfile

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [3]:
URL_T = 'https://raw.githubusercontent.com/EdwardMMiller/Data-Mining-Project---Fall-2022/main/True.csv'
URL_F = "https://raw.githubusercontent.com/EdwardMMiller/Data-Mining-Project---Fall-2022/main/Fake.csv"
# getting data from Github

True_df = pd.read_csv(URL_T)
Fake_df = pd.read_csv(URL_F)
# getting the csv files into dataframe

print("Number of rows in True_df = ", True_df.shape[0])
print("Number of rows in Fake_df = ", Fake_df.shape[0])
print("Number of columns in True_df = ", True_df.shape[1])
print("Number of columns in Fake_df = ", Fake_df.shape[1])
# Checking number of rows and columns in each dataset

columns_list_T = True_df.columns.tolist()
columns_list_F = Fake_df.columns.tolist()
print("List of columns in True_df", columns_list_T)
print("List of columns in False_df", columns_list_T)
# checking to see if columns are the same
#%%
def data_file_explore(file, df):

  """
    This is a simple function designed to do surface level
    exploration of the data in a csv file. It will print the
    first five rows of a data file, number of rows and columns in the data file
    number of unique values in the data value, and the number of missing values
    in the data file. It also inputs the minimum, maximum, mean and median
    for the numeric columns, and then finds the mode for the column along with
    the number of times it appears in that column. Finally, it also gets the
    most frequent string found for the string columns along with the number of
    times that string is found in the column. It is assumed that the data
    that is being worked with is either numeric or a string.

  Args:
    file = string referencing the filename
    df = a data frame created from reading in the file
  Returns:
    none
  """
  
  print("*********************** FILE NAME: %s ***********************\n" %file)

  # This is just a file header

  print("First five rows in file: %s\n" %file)
  display(df.head())
  # looking at the first five rows of the data

  print("\nData types present in file: %s\n" %file)
  print(df.dtypes)
  # looking at data types found in the dataframe

  print("\nNumber of rows and columns in file: %s\n" %file)
  display(df.shape)
  # Getting the size of the dataset

  print("\nNumber of unique values for each column in file: %s\n" %file)
  display(df.nunique())
  # Looking at number of unique values

  print("\nCount of missing values in file: %s\n" %file)
  display(df.isnull().sum())
  print("")
  # Counting the missing values in the datasets

  for col in df.select_dtypes(include=np.number):
    # Looping through numeric columns in data frame a
      #print("Min for col %s = %s" %(col, df[col].min()))
      #print("Max for col %s = %s" %(col, df[col].max()))
    print("Numerical Stats for column - '%s' " % col)
    print("--------------------------------------------")
    (print("Min: %s Mean: %s"
           %(df[col].min(), df[col].mean() )))
    print("Max: %s Median: %s" %(df[col].max(), df[col].median()))
    # Printing the minimum, maximum, mean and median

    col_mode = df[col].value_counts().idxmax()
    # get the most frequent value in the column
    freq = df[col].value_counts()[col_mode]
    # get the count of the most frequent value in the column

    if freq > 1:
      print("Most frequent value: %s found in column %s times.\n" %(col_mode,freq))
    else:
      print("No value repetitions found in column\n")
      # Only returning the most frequent value if it appears more than once

  for col in df.select_dtypes(include=object):
      # Only looking at columns with strings now
    print("Frequency counts for string column - '%s' " % col)
    print("--------------------------------------------------")
    col_mode = df[col].value_counts().idxmax()
      # get the most frequent value in the column
    freq = df[col].value_counts()[col_mode]
      # get the count of the most frequent value in the column
    if freq > 1:
      (print("Most frequent string value: '%s' found in column %s times.\n"
               %(col_mode,freq)))
    else:
      print("No value repetitions found in column\n")
        # Only returning the most frequent value if it appears more than once

data_file_explore('True.csv', True_df)
data_file_explore('Fake.csv', Fake_df)


Number of rows in True_df =  21417
Number of rows in Fake_df =  23481
Number of columns in True_df =  4
Number of columns in Fake_df =  4
List of columns in True_df ['title', 'text', 'subject', 'date']
List of columns in False_df ['title', 'text', 'subject', 'date']
*********************** FILE NAME: True.csv ***********************

First five rows in file: True.csv



,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"



Data types present in file: True.csv

title      object
text       object
subject    object
date       object
dtype: object

Number of rows and columns in file: True.csv



(21417, 4)


Number of unique values for each column in file: True.csv



...


Number of unique values for each column in file: Fake.csv



title      17903
text       17455
subject        6
date        1681
dtype: int64


Count of missing values in file: Fake.csv



title      0
text       0
subject    0
date       0
dtype: int64


Frequency counts for string column - 'title' 
--------------------------------------------------
Most frequent string value: 'MEDIA IGNORES Time That Bill Clinton FIRED His FBI Director On Day Before Vince Foster Was Found Dead' found in column 6 times.

Frequency counts for string column - 'text' 
--------------------------------------------------
Most frequent string value: ' ' found in column 626 times.

Frequency counts for string column - 'subject' 
--------------------------------------------------
Most frequent string value: 'News' found in column 9050 times.

Frequency counts for string column - 'date' 
--------------------------------------------------
Most frequent string value: 'May 10, 2017' found in column 46 times.



## OVERVIEW OF DATA SETS

A cursory glance at the data shows that there are no missing values in either data set and that both column types
are the same along with matching data types. There are no numerical columns her, only text and dates. However, the author of the original project points out that the **True.csv** file  shows **WASHINGTON (Reuters) - ** or **SEATTLE/WASHINGTON (Reuters)** before the news articles. Also, there are **6** unique values for **'subject'** in the **Fake.csv** file and only **2** unique values for **'subject'** in the **True.csv**, which might cause the model while training the dataset to mainly look for these two things to determine whether the news article is fake. We will want to remove these things in order to ensure that it's not so obvious which one is which, and also ensure that the model will be able to work when classifying news stories that do not have this format, as one could then easily add the <Reuters> term to a fake news story, in order to trick the model into classifying it is as true.

In [4]:
cnt = 0
for row in True_df.index:
  if "(Reuters)" in True_df.loc[row]['text']:
     cnt = cnt + 1
print("Count of rows in True_df containing term (Reuters) = %s"% cnt )
print("Number of rows in True_df data frame = ", True_df.shape[0])
print("Percentage of rows in True_df containing (Reuters) = %s"% (cnt/True_df.shape[0]*100) )
print('')
# Counting the number of instances that contain the string "(Reuters)" in True_df

cnt = 0
for row in Fake_df.index:
  if "(Reuters)" in Fake_df.loc[row]['text']:
     cnt = cnt + 1
print("Count of rows in Fake_df containing term (Reuters) = %s"% cnt )
print("Number of rows in Fake_df data frame = ", Fake_df.shape[0])
print("Percentage of rows in Fake_df containing (Reuters) = %s"% (cnt/Fake_df.shape[0]*100) )
# Counting the number of instances that contain the string "(Reuters)" in Fake_df

Count of rows in True_df containing term (Reuters) = 21247
Number of rows in True_df data frame =  21417
Percentage of rows in True_df containing (Reuters) = 99.20623803520567

Count of rows in Fake_df containing term (Reuters) = 9
Number of rows in Fake_df data frame =  23481
Percentage of rows in Fake_df containing (Reuters) = 0.038328861632809505


As the original author also pointed out, this is something that needs to be addressed with the data before putting it into the model, as one can clearly see that over **99%** of the **True_df** rows contain the term **'(Reuters)'** and under **4%** of the rows in Fake_df contain this term. Let us also look at the 'subjects' column.

In [5]:
display(True_df['subject'].unique())
display(Fake_df['subject'].unique())
# looking at the unique values in the 'subjects' column

array(['politicsNews', 'worldnews'], dtype=object)

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)

Using the **'subjects'** column in the model would be a dead give-away to the model as the unique values in the **Fake_df** and **True_df** are totally different and seeing one subject or the other would train the model to look for that subject when classifying the news article as real or fake.  Therefore, the text data needs to be pre-processed and the subject column needs to be removed as well. Also, in order to get a better look at just the text itself to see if a model can determine which class it belongs to, the date column will be removed as well.

In [6]:
def text_clean(string):
  """
  This function removes this pattern "^[A-Z/]+ \(Reuters\) - "
  from a text string
  :param string: A single string
  :return:  A single string with the pattern removed
  """
  reuters_pattern = "^[A-Z/]+ \(Reuters\) - "
  # String to remove at the start of True_df looks like
  # WASHINGTON (Reuters) - so saving this pattern
  return re.sub(reuters_pattern, "", string)
##########################################################################
test_string1 = True_df.loc[0]['text']
print("String before text_clean function applied\n")
print(test_string1)
print('')
# getting a test string to test function

print("String after text_clean function applied\n")
print(text_clean(test_string1))
print('')
# testing function to see if it removes string

def df_pre_processor(df_t, df_f, Shuffle = True):
  """ This is a basic pre-processing function that takes in
  the two dataframes True_df and Fake_df, adds correct labels
  to each, combines them, cleans the text column, removes unneeded
  columns and shuffles the dataframe

  :param df_t: a dataframe containing all true news articles
  :param df_f: a dataframe containing all fake news articles
  :param  Shuffle: bool = True
  :return: a pre-processed data frame combined from df_t & df_f
  """
  df_t['label'] = True
  df_f['label'] = False
  # adding correct labels to each

  df = pd.concat([df_t, df_f], axis=0)
  df['text'] = df['text'] .apply(text_clean)
  # combining both data frames and
  # applying text clean to the text column

  df = df.drop(["subject", 'date'],axis = 1)
  # removing two columns from df

  if Shuffle:
     df = shuffle(df).reset_index(drop=True)
  # shuffles the dataframe before returning it
  return df
##########################################################################
df_combined = df_pre_processor(True_df.copy(), Fake_df.copy())
# putting both dataframes in pre-processing function
display(df_combined.head())
# looking at new data frame
bool_test = True_df.shape[0] \
            + Fake_df.shape[0] \
            == df_combined.shape[0]
print("Will print %sTrue%s if total rows are equal - %s" % ("'","'",bool_test))
# checking to make sure new data frame has correct dimensions

String before text_clean function applied

WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary

,title,text,label
0,"Arizona man challenges primary results, allegi...",An Arizona man filed a lawsuit on Friday chall...,True
1,#WeAreNotThis: Twitter DESTROYS NC Gov. And G...,"North Carolina, long seen as the one state in ...",False
2,Australia's famed Uluru outback monolith to be...,"Australia s world-famous Uluru, formerly known...",True
3,BREAKING #WAR: Police Drag Radical Hecklers Fr...,Americans are 100% FED Up! with the radical ba...,False
4,Senior U.S. officials recommend removal of NSA...,The heads of the Pentagon and the U.S. intelli...,True


Will print 'True' if total rows are equal - True


We now have a dataframe that can be used for the next steps. Our next step is to separate the data in train and test sections and the labels will need to be separated as well.  

In [7]:
columns_list = df_combined.columns[::].tolist()
# saving columns list
train_x, test_x, train_y, test_y = train_test_split(df_combined[columns_list],
                                                    df_combined['label'],
                                                    test_size=0.2,
                                                    random_state=5)
# Splitting data in train and test groups
display(train_x)
display(train_y)
display(test_x)
display(test_y)

,title,text,label
41124,Trump administration starts countdown to NAFTA...,The Trump administration on Thursday set the c...,True
22642,Possible explosion detected near Argentine sub...,"MAR DEL PLATA, Argentina (Reuters) - The Argen...",True
5161,New Analysis Finds SIGNIFICANT Voting Anomali...,It turns out that there are a few new reasons ...,False
25434,"Calm Down, Politifact Confirms Majority Of Mo...","Recently, liberal firebrands like George Cloon...",False
2009,"Ivory Coast to retire 1,000 soldiers to slim d...",Ivory Coast will cut its armed forces by about...,True
...,...,...,...
5520,"Under scrutiny, Trump decides to dissolve his ...",(Reuters) - U.S President-elect Donald Trump s...,True
35814,LIBERAL MEDIA IGNORES MELANIA’S Visit To Home ...,"First Lady Melania Trump visits HomeSafe, phot...",False
20463,U.S. will not interfere in EU trade with Iran:...,The United States does not aim to impede Europ...,True
18638,Obama’s Gas-Guzzling Motorcade To Paris Climat...,Are you sitting down? The phony baloney Climat...,False


41124     True
22642     True
5161     False
25434    False
2009      True
         ...  
5520      True
35814    False
20463     True
18638    False
35683     True
Name: label, Length: 35918, dtype: bool

,title,text,label
10128,THE ULTIMATE HYPOCRITE: Who Are The Billionair...,While Bernie Sanders has pitched himself as th...,False
43165,IS OUR FIRST AMENDMENT RIGHT Being Stolen By T...,***WARNING***This video is GRAPHIC!***,False
14627,HOLLYWOOD’S SLICK PRO-HILLARY AD Gets A TRUMP ...,A bunch of celebrities set out to hijack our e...,False
17654,TALENTLESS Gigi Hadid Makes Ridiculous “Apolog...,Here s Gigi Hadid s ridiculous apology to Mel...,False
28412,Racist Florida Woman Tells Black People ‘You ...,A racist white woman in Florida is surely furi...,False
...,...,...,...
16304,Australian police cut chains from immigration ...,Australian police on Monday used metal cutters...,True
24306,Serial Plagiarist Does Victory Dance Over Whit...,White people the only lives that DON T matter....,False
44244,HOW HILLARY Reminded Us Last Week That We’re S...,Clinton sent out a taunting tweet of 3-0 aft...,False
27014,"Egypt's president backs Saudi purge, urges de-...","SHARM AL-SHEIKH, Egypt (Reuters) - Egyptian Pr...",True


10128    False
43165    False
14627    False
17654    False
28412    False
         ...  
16304     True
24306    False
44244    False
27014     True
4424     False
Name: label, Length: 8980, dtype: bool

After separating the data into **80% train** and **20% test** sets, the next step is get the data into a numerical form that can be recognized by the neural network. 

# New Section